# Coarse-graining a stiff macromolecule:

- Generate an ensemble of worm-like macromolecules

Consider a worm-like chain that is comprised of $N$ beads, $r_{i}$ with $i$ = 0, · · · , $N − 1$, or, equivalently, $N − 1$ bonds, $u_{i} = r_{i} − r_{i−1}$ with $i = 1$,··
· ,$N − 1$. The length of each bond is fixed, $\left\lvert u_{i}\right\rvert  = l_{0}$, and there acts a bond-angle potential between successive bonds along the molecular backbone. The energy of a chain configuration takes the form

$$\frac{\mathcal{H}(\{\mathbf{u}\})}{k_{\mathrm{B}} T}=-\frac{\epsilon_0}{l_0^2} \sum_{i=2}^{N-1} \mathbf{u}_i \cdot \mathbf{u}_{i-1}=-\epsilon_0 \sum_{i=2}^{N-1} \hat{\mathbf{u}}_i \cdot \hat{\mathbf{u}}_{i-1}$$



where $\hat{\mathbf{u}}_i$ denotes the direction of bond, $i$. Upon increasing the stiffness parameter, $\epsilon_0$, we favor macromolecules with straight bond angles, i.e., more extended configurations. The probability of a chain configuration, $\left\{\mathbf{u}_i\right\}$, is given by the Boltzmann weight

$$
P_0(\{\mathbf{u}\})=\frac{\exp \left(-\frac{\mathcal{H}\left(\left\{\mathbf{u}_i\right\}\right)}{k_{\mathrm{B}} T}\right)}{\mathcal{Z}}=\frac{1}{\mathcal{Z}} \delta\left(\left|\mathbf{u}_{\mathbf{1}}\right|-l_0\right) \prod_{i=2}^{N-1} e^{\frac{\epsilon_0}{l_0^2} \mathbf{u}_i \cdot \mathbf{u}_{i-1}} \delta\left(\left|\mathbf{u}_{\mathbf{i}}\right|-l_0\right)
$$
where $\mathcal{Z}$ is the normalization (denoted as partition function).


[hints: For the numerics set, $l_0=1$, i.e., measure all length scales in units of $l_0$. Use the parameters, $N=512$ and $\epsilon_0=5$, and generate $n=1048576$ independent molecular conformations.]


a) Generate a large sample of molecular configurations according to Equation 2
- Generate chain configurations by starting with $\mathbf{r}_0=\mathbf{0}$ and $\mathbf{r}_1=\mathbf{u}_1$ uniformly distributed on the sphere with radius, $l_0$. Given $\mathbf{u}_{i-1}$, the probability of $\mathbf{u}_i$ takes the form
$$\begin{aligned}
P\left(\mathbf{u}_i \mid \mathbf{u}_{i-1}\right) \mathrm{d}^3 \mathbf{u}_i & \sim e^{\epsilon_0 \cos \theta} \delta\left(\left|\mathbf{u}_i\right|-l_0^2\right) u_i^2 \mathrm{~d} u_i \mathrm{~d} \cos \theta \mathrm{d} \phi \\
& =\frac{\epsilon_0 e^{\epsilon_0 \cos \theta} \mathrm{d} \cos \theta}{2 \sinh \epsilon_0} \frac{\mathrm{d} \phi}{2 \pi}
\end{aligned}$$ 
where $\theta$ denotes the angle between $\mathbf{u}_i$ and $\mathbf{u}_{i-1}$ (or $l_0 \cos \theta$ is the projection of $\mathbf{u}_i$ onto the $\mathbf{u}_{i-1}$-axis), and $\phi$ is the rotation angle around the $\mathbf{u}_{i-1}$-axis. Describe how you generate the configurations.

- (alternative or bonus if both techniques are used) Write a Monte-Carlo simulation program that employs importance sampling of macromolecular configurations according to Equation 2. For example, use the snake algorithm  by choosing a chain end at random, and try to add a bead at the chosen end and remove a bead from the opposite end. Write down the detailed-balance condition to derive the acceptance criterion.

In [2]:
%%bash
/root/vsc_projects/PhyforDS/UniKurs_Phy-for-DS/Softmatter/savegh.sh

bash: line 1: /root/vsc_projects/PhyforDS/UniKurs_Phy-for-DS/Softmatter/savegh.sh: Permission denied


CalledProcessError: Command 'b'/root/vsc_projects/PhyforDS/UniKurs_Phy-for-DS/Softmatter/savegh.sh\n'' returned non-zero exit status 126.

In [ ]:
%%bash
pip install numpy multiprocessing

In [ ]:
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

def generate_configuration(_):
    # Initialize chain
    r = np.zeros((N, 3))
    u = np.zeros((N - 1, 3))

    # Generate u_1 uniformly distributed on a sphere
    theta = np.arccos(2 * np.random.random() - 1)
    phi = 2 * np.pi * np.random.random()
    u[0] = np.array([np.sin(theta) * np.cos(phi),
                     np.sin(theta) * np.sin(phi),
                     np.cos(theta)])

    # Generate subsequent bonds
    for i in range(1, N - 1):
        xi = np.random.random()
        cos_theta = np.log(1 - xi * np.sinh(epsilon_0)) / epsilon_0
        theta = np.arccos(cos_theta)
        phi = 2 * np.pi * np.random.random()
        u_i = np.array([np.sin(theta) * np.cos(phi),
                        np.sin(theta) * np.sin(phi),
                        np.cos(theta)])
        u[i] = u_i

    return u


# Parameters
N = 512
epsilon_0 = 5
sample_size = 1048576

# Generate molecular configurations in parallel with progress bar
with Pool() as p, tqdm(total=sample_size) as pbar:
    configurations = []
    for config in p.imap_unordered(generate_configuration, range(sample_size)):
        configurations.append(config)
        pbar.update(1)

# Access individual configurations
print(configurations[0])  # Example: Print the first configuration


In [ ]:
# Save configurations to a file
filename = "molecular_configurations.npy"
np.save(filename, configurations)
print(f"Saved {len(configurations)} configurations to {filename}.")

In [19]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from multiprocessing import Pool
from tqdm import tqdm

def generate_configuration(_):
    # Initialize chain
    r = np.zeros((N, 3))
    u = np.zeros((N - 1, 3))

    # Generate u_1 uniformly distributed on a sphere
    theta = np.arccos(2 * np.random.random() - 1)
    phi = 2 * np.pi * np.random.random()
    u[0] = np.array([np.sin(theta) * np.cos(phi),
                     np.sin(theta) * np.sin(phi),
                     np.cos(theta)])

    # Generate subsequent bonds
    for i in range(1, N - 1):
        xi = np.random.random()
        sinh_val = np.sinh(epsilon_0)
        if xi * sinh_val >= 1:  # To prevent getting invalid input for np.log
            xi = 1. / sinh_val
        cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
        theta = np.arccos(cos_theta)
        phi = 2 * np.pi * np.random.random()
        u_i = np.array([np.sin(theta) * np.cos(phi),
                        np.sin(theta) * np.sin(phi),
                        np.cos(theta)])
        u[i] = u_i

    return u


# Parameters
N = 512
epsilon_0 = 5
sample_size = 1048576

# Generate molecular configurations in parallel with progress bar
with Pool() as p, tqdm(total=sample_size) as pbar:
    configurations = []
    for config in p.imap_unordered(generate_configuration, range(sample_size)):
        configurations.append(config)
        pbar.update(1)

# Access individual configurations
print(configurations[0])  # Example: Print the first configuration



# Save configurations to a file
filename = "molecular_configurations_new.npy"
np.save(filename, configurations)
print(f"Saved {len(configurations)} configurations to {filename}.")


  0%|          | 0/1048576 [00:00<?, ?it/s]/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1 - xi * sinh_val) / epsilon_0
/tmp/ipykernel_12336/3035712178.py:24: RuntimeWarning: divide by zero encountered in log
  cos_theta = np.log(1

KeyboardInterrupt: 

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from multiprocessing import Pool
from tqdm import tqdm

def generate_configuration(_):
    # Initialize chain
    r = np.zeros((N, 3))
    u = np.zeros((N - 1, 3))

    # Generate u_1 uniformly distributed on a sphere
    theta = np.arccos(2 * np.random.random() - 1)
    phi = 2 * np.pi * np.random.random()
    u[0] = np.array([np.sin(theta) * np.cos(phi),
                     np.sin(theta) * np.sin(phi),
                     np.cos(theta)])

    # Generate subsequent bonds
    for i in range(1, N - 1):
        theta = np.random.vonmises(0, epsilon_0)  # Generate theta from von Mises distribution
        cos_theta = np.cos(theta)
        phi = 2 * np.pi * np.random.random()
        rot = R.from_rotvec(theta * u[i - 1])
        u[i] = rot.apply([np.sin(phi), 0, cos_theta])

    return u


# Parameters
N = 512
epsilon_0 = 5
sample_size = 1048576

# Generate molecular configurations in parallel with progress bar
with Pool() as p, tqdm(total=sample_size) as pbar:
    configurations = []
    for config in p.imap_unordered(generate_configuration, range(sample_size)):
        configurations.append(config)
        pbar.update(1)

# Access individual configurations
print(configurations[0])  # Example: Print the first configuration



# Save configurations to a file
filename = "molecular_configurations_newer.npy"
np.save(filename, configurations)
print(f"Saved {len(configurations)} configurations to {filename}.")


---------

b) Calculate averages and statistical errors of the following quantities and compare them to the exact results (where available)
- average bond-angle
$$
\langle\cos \theta\rangle=\frac{\left(\epsilon_0-1\right) e^{\epsilon_0}+\left(\epsilon_0+1\right)^{-\epsilon_0}}{2 \epsilon_0 \sinh \epsilon_0}=\frac{1-\frac{1}{\epsilon_0}+\left(1+\frac{1}{\epsilon_0}\right) e^{-2 \epsilon_0}}{1-e^{-2 \epsilon_0}}
$$
- bond-angle correlation along the macromolecule
$$
C(\Delta i)=\frac{\left\langle\mathbf{u}_i \cdot \mathbf{u}_{i+\Delta i}\right\rangle}{l_0^2}=\langle\cos \theta\rangle^{|\Delta i|}=e^{-\frac{l_0|\Delta i|}{l_p}} \quad \text { with } l_p=-\frac{l_0}{\ln \langle\cos \theta\rangle}
$$
where $l_p$ is called the persistence length.
- mean-square end-to-end distance, $\left\langle R_{\mathrm{EE}}^2\right\rangle \equiv\left\langle\left(\mathbf{r}_{N-1}-\mathbf{r}_0\right)^2\right\rangle=\left\langle\left(\sum_{i=1}^{N-1} \mathbf{u}_i\right)^2\right\rangle$
$$
\begin{aligned}
& \left\langle R_{\mathrm{EE}}^2\right\rangle=(N-1) l_0^2\left(\frac{1+\langle\cos \theta\rangle}{1-\langle\cos \theta\rangle}-\frac{2\langle\cos \theta\rangle}{N-1} \frac{1-\langle\cos \theta\rangle^{N-1}}{(1-\langle\cos \theta\rangle)^2}\right) \\
& \stackrel{l_p \gg l_0}{\approx} 2 L l_p\left(1-\frac{l_p}{L}\left[1-e^{-L / l_p}\right]\right) \\
&
\end{aligned}
$$
where $L=(N-1) l_0$ denotes the contour length of the chain molecules.



In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from multiprocessing import Pool
from tqdm import tqdm
from sklearn.utils import resample

def calculate_averages(configurations):
    num_configurations = len(configurations)
    cos_thetas = []
    u_dot_u_deltas = []
    R_EE_squares = []

    # Calculate quantities for each configuration
    for config in configurations:
        # Calculate cos_theta for each bond
        cos_theta = np.sum(config[:-1] * config[1:], axis=1)
        cos_thetas.append(cos_theta)

        # Calculate u_i.u_{i+Delta_i} for each pair of bonds
        u_dot_u_delta = np.sum(config[:-2] * config[2:], axis=1)
        u_dot_u_deltas.append(u_dot_u_delta)

        # Calculate R_EE^2 for the chain
        R_EE_square = np.sum((config[-1] - config[0])**2)
        R_EE_squares.append(R_EE_square)

    cos_thetas = np.concatenate(cos_thetas)
    u_dot_u_deltas = np.concatenate(u_dot_u_deltas)

    # Calculate averages
    average_cos_theta = np.mean(cos_thetas)
    average_u_dot_u_delta = np.mean(u_dot_u_deltas)
    average_R_EE_square = np.mean(R_EE_squares)

    # Estimate errors using bootstrap resampling
    cos_theta_err = np.std(resample(cos_thetas, replace=True, n_samples=num_configurations))
    u_dot_u_delta_err = np.std(resample(u_dot_u_deltas, replace=True, n_samples=num_configurations))
    R_EE_square_err = np.std(resample(R_EE_squares, replace=True, n_samples=num_configurations))

    return average_cos_theta, cos_theta_err, average_u_dot_u_delta, u_dot_u_delta_err, average_R_EE_square, R_EE_square_err

average_cos_theta, cos_theta_err, average_u_dot_u_delta, u_dot_u_delta_err, average_R_EE_square, R_EE_square_err = calculate_averages(configurations)

print(f"Average cos(theta) = {average_cos_theta} ± {cos_theta_err}")
print(f"Average u_i.u_{i+Delta_i} = {average_u_dot_u_delta} ± {u_dot_u_delta_err}")
print(f"Average R_EE^2 = {average_R_EE_square} ± {R_EE_square_err}")


# Calculate averages and statistical errors
average_cos_theta, cos_theta_err, average_u_dot_u_delta, u_dot_u_delta_err, average_R_EE_square, R_EE_square_err = calculate_averages(configurations)

# Save results to a text file
with open('averages_and_errors.txt', 'w') as f:
    f.write(f"Average cos(theta) = {average_cos_theta} ± {cos_theta_err}\n")
    f.write(f"Average u_i.u_{i+Delta_i} = {average_u_dot_u_delta} ± {u_dot_u_delta_err}\n")
    f.write(f"Average R_EE^2 = {average_R_EE_square} ± {R_EE_square_err}\n")

print("Averages and errors have been saved to 'averages_and_errors.txt'")


----------

c) Generate a joint sequence of the polymer's square end-to-end distance, $R_{\mathrm{EE}}^2(t)$, and the bond-angle energy, $\mathcal{H}(\{\mathbf{u}(t)\})$, according to Equation 1. $t$ denotes the index of the configuration in the sequence. Use single-histogram extrapolation
$$
\begin{aligned}
\left\langle R_{\mathrm{EE}}^2\right\rangle_{\epsilon_0^{\prime}} & =\frac{\left\langle e^{\frac{\left(\epsilon_0^{\prime}-\epsilon_0\right)}{l_0^2} \sum_i \mathbf{u}_i \cdot \mathbf{u}_{i-1}} R_{\mathrm{EE}}^2(\{\mathbf{u}\})\right\rangle_\epsilon}{\left\langle e^{\frac{\left(\epsilon_0^{\prime}-\epsilon_0\right)}{l_0^2} \sum_i \mathbf{u}_i \cdot \mathbf{u}_{i-1}}\right\rangle_\epsilon} \\
& \approx \frac{\sum_t e^{\frac{\left(\epsilon_0^{\prime}-\epsilon_0\right)}{l_0^2} \sum_i\left[\mathbf{u}_i(t) \cdot \mathbf{u}_{i-1}(t)-c\right]} R_{\mathrm{EE}}^2(\{\mathbf{u}(t)\})}{\sum_t e^{\frac{\left(\epsilon_0^{\prime}-\epsilon_0\right)}{l_0^2} \sum_i\left[\mathbf{u}_i(t) \cdot \mathbf{u}_{i-1}(t)-c\right]}}
\end{aligned}
$$
to validate your sample of generated configurations by comparing the meansquare end-to-end distance at two, neighboring values of the stiffness parameter, $\epsilon_0^{\prime}$ and $\epsilon_0$. In the last step we have approximated the integral over configuration space by the sum over the sequence. $c \approx\left\langle\mathbf{u}_i \cdot \mathbf{u}_{i-1}\right\rangle=l_0^2\langle\cos \theta\rangle$ is a constant to avoid over/underflow issues.
[hint: choose $\epsilon^{\prime}=\epsilon \pm 0.25$ ]
- Coarse grain the macromolecular configurations
by defining new, coarse-grained bead positions and bond vectors, according to
$$
\mathbf{R}_J=\mathbf{r}_{(J+1 / 2) 2^\lambda} \quad \text { and } \quad \mathbf{U}_J=\mathbf{R}_J-\mathbf{R}_{J-1}=\sum_{i=J 2^\lambda}^{(J+1) 2^\lambda-1} \mathbf{u}_i
$$